In [1]:
# --- Imports ---
import os
import io
import base64
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from textwrap import shorten

# --- Load API key from .env ---
load_dotenv("C:\\EUacademy\\.env")  # adjust path if needed
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file.")

# Initialize OpenAI client (new SDK style)
from openai import OpenAI
client = OpenAI(api_key=api_key)

# --- CSV loader ---
def load_cycles_csv(path="./physiological_cycles_today.csv"):
    if not os.path.exists(path):
        raise FileNotFoundError(f"CSV not found: {os.path.abspath(path)}")
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    return df

# --- Summary & charts ---
def analyze_df(df):
    averages = {
        "Recovery": round(df["Recovery_score_"].mean(), 2),
        "Rest HR": round(df["Resting_heart_rate_(bpm)"].mean(), 2),
        "HRV": round(df["Heart_rate_variability_(ms)"].mean(), 2),
        "Sleep Perf": round(df["Sleep_performance_"].mean(), 2),
        "Sleep Debt": round(df["Sleep_debt_(min)"].mean(), 2) if "Sleep_debt_(min)" in df else 0,
    }
    plt.figure(figsize=(7,3))
    plt.bar(list(averages.keys()), list(averages.values()))
    plt.title("Average Key Metrics")
    plt.show()

    recovery_dist = {
        "Low": int((df["Recovery_score_"] < 50).sum()),
        "Medium": int(((df["Recovery_score_"] >= 50) & (df["Recovery_score_"] < 80)).sum()),
        "High": int((df["Recovery_score_"] >= 80).sum())
    }
    print("Recovery Score Distribution:", recovery_dist)

    summary_text = [
        f"Days: {len(df)}",
        f"Avg Recovery: {averages['Recovery']}",
        f"Avg Rest HR: {averages['Rest HR']}",
        f"Avg HRV: {averages['HRV']}",
        f"Avg Sleep Performance: {averages['Sleep Perf']}",
        f"Recovery Distribution: {recovery_dist}"
    ]
    if "Sleep_debt_(min)" in df:
        avg_sleep_debt = round(df["Sleep_debt_(min)"].mean(), 2)
        summary_text.append(f"Avg Sleep Debt: {avg_sleep_debt}")
    return "\n".join(summary_text)

# --- Chat helper ---
def chat_once(prompt, context_summary, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": "You are a health data analyst, concise and actionable."},
        {"role": "user", "content": f"Dataset summary:\n{context_summary}\n\nUser prompt: {prompt}"}
    ]
    resp = client.chat.completions.create(model=model, messages=messages, temperature=0.2)
    return resp.choices[0].message.content.strip()

# --- Run once ---
csv_path = input("Enter CSV path (press Enter for default './physiological_cycles_today.csv'): ").strip() or "./physiological_cycles_today.csv"
df = load_cycles_csv(csv_path)
context_summary = analyze_df(df)

print("\n💬 Chat mode (type 'q' to quit)")
while True:
    q = input("\nYour prompt> ").strip()
    if q.lower() == "q":
        print("Bye!")
        break
    if q:
        try:
            ans = chat_once(q, context_summary)
            print("\nAssistant:", ans)
        except Exception as e:
            print("[Error]", e)


ValueError: OPEN_API_KEY not found in .env file.